In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Uterine_Corpus_Endometrial_Carcinoma/GSE32507'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profile of carcinosarcoma (CS), endometrioid adenocarcinoma (EC) and sarcoma (US) of uterine corpus"
!Series_summary	"To examine the simlarity of CS, EC and US, we performed microarray analysis of frozen tissues of 46 patients (14 CS, 24 EC and 8 US)."
!Series_overall_design	"Frozen tissues of 46 patients (14CS, 24EC and 8US) were subjected to cDNA microarray analysis."
Sample Characteristics Dictionary:
{0: ['tissue: carcinosarcoma', 'tissue: endometrioid adenocarcinoma', 'tissue: sarcoma'], 1: ['carcinosarcoma status: : heterologous', 'carcinosarcoma status: : homologous', nan]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Checking gene expression data availability
# Based on !Series_overall_design mentioning cDNA microarray analysis, gene expression data is likely available
is_gene_available = True

# Variable availability from the sample characteristics
sample_characteristics_dict = {
    0: ['tissue: carcinosarcoma', 'tissue: endometrioid adenocarcinoma', 'tissue: sarcoma'], 
    1: ['carcinosarcoma status: : heterologous', 'carcinosarcoma status: : homologous', float('nan')]
}

# Determine trait_row by checking the sample characteristics related to 'Uterine_Corpus_Endometrial_Carcinoma'
if 0 in sample_characteristics_dict and len(set(value.split(":")[1].strip() for value in sample_characteristics_dict[0] if value )) > 1:
    trait_row = 0
else:
    trait_row = None

# Age and Gender data are not present in the provided characteristics dictionary
age_row = gender_row = None

# Data type conversion functions
def convert_trait(value):
    # Extract the value after the ':' and determine type based on unique values (classification)
    if value is None:
        return None
    value = value.split(":")[1].strip() if ":" in value else value
    if value == 'carcinosarcoma':
        return 1  # Assuming 1 for presence of carcinoma
    elif value == 'endometrioid adenocarcinoma' or value == 'sarcoma':
        return 0  # Assuming 0 for absence of carcinoma
    return None

convert_age = convert_gender = None  # As age and gender rows are not available

# Saving metadata
save_cohort_info('GSE32507', './preprocessed/Uterine_Corpus_Endometrial_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction should not be performed here as trait_row is None 


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# Based on biomedical conventions, headers such as 'A_23_P100001', 'A_23_P100011' etc., are microarray probe IDs.
# No human gene symbols detected in the given headers which primarily describe experimental conditions or probes.


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(+)eQC-39', '(+)eQC-41',
       '(+)eQC-42', '(-)3xSLv1', 'A_23_P100001', 'A_23_P100011',
       'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100092'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Identify the keys for gene mapping
    identifier_key = 'ID'
    gene_symbol_key = 'GENE_SYMBOL'

    # 2. Get the dataframe for gene to probe ID mapping
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping to convert from probe IDs to gene symbols
    gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:

# For STEP 2 (if `trait_row` was None erroneously):
if trait_row is not None:
    # Define the conversion functions
    def convert_trait(value):
        if value is None:
            return None
        value = value.split(":")[1].strip() if ":" in value else value
        if value == 'carcinosarcoma':
            return 1  # Assuming 1 for presence of carcinoma
        elif value == 'endometrioid adenocarcinoma' or value == 'sarcoma':
            return 0  # Assuming 0 for absence of carcinoma
        return None
    
    convert_age = convert_gender = None  # As age and gender rows are not available
    
    # Extract clinical features
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Uterine_Corpus_Endometrial_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Uterine_Corpus_Endometrial_Carcinoma/trait_data/GSE32507.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))

# Proceeding with STEP 7
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Uterine_Corpus_Endometrial_Carcinoma/gene_data/GSE32507.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Uterine_Corpus_Endometrial_Carcinoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE32507', './preprocessed/Uterine_Corpus_Endometrial_Carcinoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Uterine_Corpus_Endometrial_Carcinoma/GSE32507.csv'
    unbiased_merged_data.to_csv(csv_path)


{'GSM804806': [1], 'GSM804807': [1], 'GSM804808': [1], 'GSM804809': [1], 'GSM804810': [0], 'GSM804811': [0], 'GSM804812': [0], 'GSM804813': [0], 'GSM804814': [0], 'GSM804815': [0], 'GSM804816': [0], 'GSM804817': [0], 'GSM804818': [0], 'GSM804819': [0], 'GSM804820': [0], 'GSM804821': [0], 'GSM804822': [0], 'GSM804823': [0], 'GSM804824': [0], 'GSM804825': [0], 'GSM804826': [0], 'GSM804827': [0], 'GSM804828': [0], 'GSM804829': [1], 'GSM804830': [1], 'GSM804831': [0], 'GSM804832': [0], 'GSM804833': [1], 'GSM804834': [1], 'GSM804835': [1], 'GSM804836': [1], 'GSM804837': [0], 'GSM804838': [0], 'GSM804839': [1], 'GSM804840': [0], 'GSM804841': [0], 'GSM804842': [0], 'GSM804843': [1], 'GSM804844': [1], 'GSM804845': [1], 'GSM804846': [0], 'GSM804847': [0], 'GSM804848': [0], 'GSM804849': [0], 'GSM804850': [0], 'GSM804851': [0]}


For the feature 'Uterine_Corpus_Endometrial_Carcinoma', the least common label is '1.0' with 14 occurrences. This represents 30.43% of the dataset.
The distribution of the feature 'Uterine_Corpus_Endometrial_Carcinoma' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Uterine_Corpus_Endometrial_Carcinoma/cohort_info.json
